In [21]:
import os
from glob import glob
import shutil
from tqdm.notebook import tqdm 
import dicom2nifti
import numpy as np
import nibabel as nib
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [22]:
training_directory="D:/MLDatasets/TCGA-GBM Datasets/Training"

In [23]:
flair_files = sorted(glob(os.path.join(training_directory, f"images_{0}", "*.nii.gz")))
t1_files = sorted(glob(os.path.join(training_directory, f"images_{1}", "*.nii.gz")))
t1Gd_files = sorted(glob(os.path.join(training_directory, f"images_{2}", "*.nii.gz")))
t2_files = sorted(glob(os.path.join(training_directory, f"images_{3}", "*.nii.gz")))
mask_files = sorted(glob(os.path.join(training_directory, f"segmentation", "*.nii.gz")))
print(len(flair_files))
print(len(t1_files))
print(len(t1Gd_files))
print(len(t2_files))
print(len(mask_files))

102
102
102
102
102


In [28]:
import torch
current_using = True 

for i in tqdm(range(len(flair_files))):
    flair_image = nib.load(flair_files[i]).get_fdata()
    t1_image = nib.load(t1_files[i]).get_fdata()
    t1Gd_image = nib.load(t1Gd_files[i]).get_fdata()
    t2_image = nib.load(t2_files[i]).get_fdata()
    combined_image = np.stack([flair_image, t1_image, t1Gd_image, t2_image], axis=3)

    mask = nib.load(mask_files[i]).get_fdata()
    mask = mask.astype(np.uint8)
    
    if(i == 0):
        print(combined_image.shape)
    
    combined_image = nib.Nifti1Image(combined_image, affine=np.eye(4))
    mask = nib.Nifti1Image(mask, affine=np.eye(4))

    if current_using == True:
        nib.save(combined_image, os.path.join(training_directory, "imagesv4", f"image_{i}.nii.gz"))
        nib.save(mask, os.path.join(training_directory, "segmentationv4", f"mask_{i}.nii.gz"))
    

  0%|          | 0/102 [00:00<?, ?it/s]

(240, 240, 155, 4)


100%|██████████| 102/102 [04:15<00:00,  2.51s/it]


In [29]:
import os
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib
from monai.transforms import (
    Activations,
    Activationsd,
    AsDiscrete,
    AsDiscreted,
    Compose,
    Invertd,
    LoadImaged,
    MapTransform,
    NormalizeIntensityd,
    Orientationd,
    RandFlipd,
    RandScaleIntensityd,
    RandShiftIntensityd,
    RandSpatialCropd,
    Spacingd,
    EnsureChannelFirstd,
    ToTensord
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism
import torch

class ConvertToMultiChannelBasedOnBratsClassesd(MapTransform):
    """
    Convert labels to multi channels based on brats classes:
    label 1 is the peritumoral edema
    label 2 is the GD-enhancing tumor
    label 3 is the necrotic and non-enhancing tumor core
    The possible classes are TC (Tumor core), WT (Whole tumor)
    and ET (Enhancing tumor).

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            result = []
            
            label1 = torch.from_numpy(d[key] == 1)
            label2 = torch.from_numpy(d[key] == 2)
            label4 = torch.from_numpy(d[key] == 4) 

            result.append( # WT
                torch.logical_or(torch.logical_or(label1, label2), label4)
            )
            
            result.append( #TC
                torch.logical_or(label1, label4)
            )
            
            result.append( #EC
                label4
            )

            d[key] = torch.stack(result, axis=0).float()
        return d

def prepare(in_dir, pixdim=(1.0, 1.0, 1.0), a_min=0, a_max=503, spatial_size=[128,128,144, 3], trainloader_batchsize=1):
    print("Preparing...")
    set_determinism(seed=0)

    path_train_volumes = sorted(glob(os.path.join(in_dir, "imagesv3", "*.nii.gz")))
    path_train_segmentation = sorted(glob(os.path.join(in_dir, "segmentationv3", "*.nii.gz")))

    '''DEBUG'''
    print(f"Length of path_train_volumes: {len(path_train_volumes)}")    
    print(f"Length of path_train_segmentation: {len(path_train_segmentation)}")    
    '''DEBUG'''
    
    train_files = [{"image": image_name, "label": label_name} for image_name, label_name in zip(path_train_volumes, path_train_segmentation)]
    
    train_transform = Compose(
        [
            LoadImaged(keys=["image", "label"]),
            AddChanneld(keys=["image", "label"]),
            EnsureChannelFirstd(keys="image"),
            ConvertToMultiChannelBasedOnBratsClassesd(keys="label"),
            Orientationd(keys=["image", "label"], axcodes="RAS"),
            Spacingd(keys=["image", "label"], pixdim=(1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
            CropForegroundd(keys=['image', 'label'], source_key='image'),
            Resized(keys=["image", "label"], spatial_size=spatial_size, mode='nearest'),   
            RandSpatialCropd(keys=["image", "label"], roi_size=spatial_size, random_size=False),
            RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=0),
            RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=1),
            RandFlipd(keys=["image", "label"], prob=0.5, spatial_axis=2),
            NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
            RandScaleIntensityd(keys="image", factors=0.1, prob=1.0),
            RandShiftIntensityd(keys="image", offsets=0.1, prob=1.0),
        ]
    )

    train_ds = CacheDataset(data=train_files, transform=train_transform,cache_rate=1.0)
    train_loader = DataLoader(train_ds, batch_size=trainloader_batchsize)
    print("Preparing done.")

    return train_loader



In [30]:

trainloader = prepare(training_directory)

Preparing...
Length of path_train_volumes: 102
Length of path_train_segmentation: 102


Loading dataset:   0%|          | 0/102 [00:01<?, ?it/s]


RuntimeError: applying transform <monai.transforms.spatial.dictionary.Orientationd object at 0x00000238816729A0>